In [1]:
from matplotlib import font_manager, rc

# 1. 한글 폰트 설정 (Windows의 '맑은 고딕' 폰트)
font_path = "c:/Windows/Fonts/malgun.ttf"  # 폰트 경로 (맑은 고딕)
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# 1. 여러 하위 폴더에서 best.pt 모델을 가져오기
model_folder = 'best_pt_정리'  # 모델들이 담긴 상위 폴더 경로
model_paths = [os.path.join(model_folder, subfolder, 'best.pt') for subfolder in os.listdir(model_folder) if os.path.isdir(os.path.join(model_folder, subfolder))]

# 2. 테스트할 이미지 파일 경로 정의
image_folder = 'test_snack_detect'
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# 3. 각 모델별 예측 결과를 저장할 리스트
all_models_processed_images = []

# 4. 각 모델별로 예측 수행
for model_path in model_paths:
    if os.path.exists(model_path):
        # 모델 불러오기
        model = YOLO(model_path)
        model_name = os.path.basename(os.path.dirname(model_path))  # 모델이 있는 폴더 이름을 모델 이름으로 사용

        # 모델의 예측 결과를 저장할 리스트
        processed_images = []

        # 각 이미지에 대해 예측 수행
        for image_file in image_files:
            image_path = os.path.join(image_folder, image_file)
            image = cv2.imread(image_path)
            resized_image = cv2.resize(image, (640, 480))  # 이미지 크기 조정

            # 모델 예측 수행
            results = model(resized_image)

            # 예측 결과에 따라 바운딩 박스와 텍스트 추가
            for result in results:
                for box in result.boxes:
                    # 바운딩 박스 좌표 추출 및 정수 변환
                    x_min, y_min, x_max, y_max = map(int, box.xyxy[0])

                    # 레이블과 신뢰도 점수 추출
                    confidence = box.conf.item()
                    label = int(box.cls.item())
                    label_name = model.names[label]  # 클래스 이름 가져오기

                    # OpenCV로 바운딩 박스와 텍스트 추가
                    cv2.rectangle(resized_image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
                    text = f'{label_name}: {confidence:.2f}'
                    cv2.putText(resized_image, text, (x_min, y_min - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

            # RGB로 변환 후 처리된 이미지를 저장
            processed_images.append((cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB), f"{model_name} - {image_file}"))

        # 모델별 처리된 이미지를 저장
        all_models_processed_images.append((model_name, processed_images))

# 5. 여러 이미지를 n행 × n열 그리드로 시각화하는 함수 정의
def plot_grid(images_with_titles, rows, cols, grid_title):
    """이미지를 n행 × n열 그리드로 표시, 그리드 제목 포함"""
    fig, axes = plt.subplots(rows, cols, figsize=(15, 15))  # 그리드 생성

    # 각 이미지에 대해 축 설정
    for i, (img, title) in enumerate(images_with_titles):
        ax = axes.flatten()[i] if i < rows * cols else None
        if ax:
            ax.imshow(img)  # 이미지 표시
            ax.axis('off')  # 축 숨기기

    # 그리드 전체의 제목 설정
    fig.suptitle(grid_title, fontsize=16)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # 제목 공간 확보
    plt.show()

# 6. 각 모델별로 그리드를 추가하여 시각화
for model_name, processed_images in all_models_processed_images:
    total_images = len(processed_images)
    cols = 5  # 한 줄에 표시할 이미지 수
    rows = (total_images // cols) + 1 if total_images % cols != 0 else total_images // cols

    # 모델의 예측 결과를 그리드로 시각화, 그리드에 모델 이름을 제목으로 설정
    plot_grid(processed_images, rows=rows, cols=cols, grid_title=f"Model: {model_name}")
